# Onboarding Use Case (1:1 Verification)
-----

Here we are going to take a look at the Onboarding case. Here a user is asked to present an image of a trusted identification document, then snap a selfie image. The following diagram details the process. 




![Onboarding](onboarding_process.png "Onboarding")

1. user presents an image of an identification document like a drivers license or passport
2. user snaps a selfie which will be used to compare to the drivers license
3. system detects a face in the identification document and performs quality checks 
4. system detects a face in the selfie and performs quality checks
5. system compares the identification image to the selfie image 
    - if the similarity is above the specified threshold then we say that the faces match 
    - if the similarity is below the specified threshold then  we say that the faces DON'T match
6. system checks selfie against known onboarded users and fraudsters. 
7. indexes the user and the user is onboarded. 



In [ ]:
import io
import boto3
import json
from IPython.display import Image as IImage
import pandas as pd

%store -r bucket_name
mySession = boto3.session.Session()
aws_region = mySession.region_name
print("AWS Region: {}".format(aws_region))
print("AWS Bucket: {}".format(bucket_name))


### Setup Clients 
-----
Here we are going to use both S3 and Rekognition apis  

In [ ]:
s3_client  = boto3.client('s3')
rek_client = boto3.client('rekognition')

## Step 0. Setup collections 
-----
Here we will setup two collections "registered-users" and "fraudulent-users". The collections will be used to ensure that the user hasn't previously registered and that the user is not a known fraudster. Here we'll quickly create the two collections and index a handful of images. 

<div class="alert alert-info"><strong> Avoiding Duplicates & Fraudsters </strong>
    
In practice, we use collections for many purposes. One is to avoid duplicate registrations; Without some form of biometric based onboarding, it is pretty easy for a user to create multiple accounts on a system simply by using different email addresses. Using a collection prevents this, because you can simply search the collection to see if a user has previously onobarded. Collections have proven to be effective at identifying and dealing with duplicate onboarding attempts as well as identifying fraudsters and bad actors. 

</div>


<div class="alert alert-info"><strong> Create two Collections </strong>

Here we are going to create two collections "idv-registered-users" and "idv-fraudulent-users" we'll load both collections with some images, and in step 6 and 7 of our process we'll check that the user hasn't previously onboarded. 

</div>

In [ ]:
## Name your Collection 

# onboarded users collection
collection_name = '' # name your collection something like "onboarded-users"

try:
    # try to delete it if it exists 
    rek_client.delete_collection(
        CollectionId=collection_name
    )
    # then create it
    rek_client.create_collection(
        CollectionId=collection_name
    )
except:
    # if it doesn't exist create it 
    response = rek_client.create_collection(
        CollectionId=collection_name
    )

# fraudulent user collection 
fraud_collection_name = ' ' # name your collection something like "fraudulent-users"

try:
    # try to delete it if it exists 
    rek_client.delete_collection(
        CollectionId=fraud_collection_name
    )
    # then create it 
    rek_client.create_collection(
        CollectionId=fraud_collection_name
    )
except:
    # if it doesn't exist create it 
    response = rek_client.create_collection(
        CollectionId=fraud_collection_name
    )

In [ ]:
# -- read the image map into a pandas dataframe --
obj = s3_client.get_object(Bucket=bucket_name, Key='IDVImageMapping.xlsx')
image_map = pd.read_excel(io.BytesIO(obj['Body'].read()))

## Onboarded useres 
dict_of_faces = image_map[["reference_name","reference_image"]].sample(n=20).to_dict('records')

for rec in dict_of_faces:
    
    response = rek_client.index_faces(
        CollectionId= collection_name,
        Image={
            'S3Object': {
                'Bucket': bucket_name,
                'Name': rec["reference_image"],
            }
        },
        ExternalImageId=rec['reference_name'],
        DetectionAttributes=[
            'DEFAULT',
        ],
        MaxFaces=1, # maximum faces detected 
        QualityFilter='AUTO' # apply the quality filter. 
        )
    face_id = response['FaceRecords'][0]['Face']['FaceId']
    print("ImageName: {}, FaceID: {}".format(rec["reference_image"], face_id))
    

print("--- indexing {} complete --- \n".format(collection_name))


## Onboarded useres 
dict_of_faces = image_map[["reference_name","reference_image"]].sample(n=20).to_dict('records')

for rec in dict_of_faces:
    
    response = rek_client.index_faces(
        CollectionId= fraud_collection_name,
        Image={
            'S3Object': {
                'Bucket': bucket_name,
                'Name': rec["reference_image"],
            }
        },
        ExternalImageId=rec['reference_name'],
        DetectionAttributes=[
            'DEFAULT',
        ],
        MaxFaces=1, # maximum faces detected 
        QualityFilter='AUTO' # apply the quality filter. 
        )
    face_id = response['FaceRecords'][0]['Face']['FaceId']
    print("ImageName: {}, FaceID: {}".format(rec["reference_image"], face_id))
    

print("--- indexing {} complete ---\n".format(fraud_collection_name))

## Step 1.  User captures image of identification document 
-----
The image below include sample image of a RealID drivers license a 

In [ ]:
## Image of a Face
id_image = "jane_maria_sc_dl.jpeg"
display(IImage(url=s3_client.generate_presigned_url('get_object', 
                                                    Params={'Bucket': bucket_name, 
                                                            'Key'   : id_image})))

## Step 2.  User snaps a selfie   
-----
The image below is a sample selfie of the same person, we'll use this to perform our comparisons and searches

In [ ]:
## Image of a Face
selfie_image = "JaneMarieSelfie.png"
display(IImage(url=s3_client.generate_presigned_url('get_object', 
                                                    Params={'Bucket': bucket_name, 
                                                            'Key'   : selfie_image})))

## Step 3. Check Identification Document 
-----

Here we want to do some basic checks on the Identification Document :
1. that we can detect a face in the drivers license  
2. the quality (sharpness and brightness) are sufficient to match with 

Note: we could do several other checks, but we'll see those in subsequent modules. 

<div class="alert alert-info"><strong> DetectFaces </strong>
    
The **DetectFaces** operation that looks for key facial features such as eyes, nose, and mouth to detect faces in an input image. Amazon Rekognition Image detects the 100 largest faces in an image.

Here we actually will detect two faces in the driver's license. 

    
The **DetectText** can be used to extract text from the drivers license as well. For our purposes we'll skip this step. 

</div>

<div class="alert alert-info"><strong> Note </strong>
    
Take a look at the FaceDetails of each face detected. it will return several helpful details of image quality for each face in the image. 

- BoundingBox
- AgeRange
- Gender
- Landmarks
- Quality
- Pose 

</div>

The drivers license image returns two faces, you can see the different faces using the following 

```python 
response['FaceDetails'][0]
response['FaceDetails'][1]
```

In [ ]:
# -- drivers license -- 
response = rek_client.detect_faces(Image={'S3Object':{
    'Bucket':bucket_name,
    'Name':id_image}},
                                   Attributes=['ALL'])
response

## Step 4. Selfie Image quality checks
----

Here we want to do some basic checks:
1. that we can detect a face in the drivers license and the selie 
2. the quality (sharpness and brightness) are sufficent to match with 

Note: we could do several other checks, but we'll see those in module 3.


<div class="alert alert-info"><strong> DetectFaces </strong>
    
The **DetectFaces** operation that looks for key facial features such as eyes, nose, and mouth to detect faces in an input image. Amazon Rekognition Image detects the 100 largest faces in an image.



</div>

<div class="alert alert-info"><strong> Note </strong>
    
Take a look at the FaceDetails of the selfie and compare it to the face details on the drivers license. make note of the quality etc. 

- BoundingBox
- AgeRange
- Gender
- Landmarks
- Quality
- Pose 

</div>

The quality of the 1st face in the drivers license is different than the quality of the face in the selfie:

```python 
# -- first in drivers license quality --
'Quality': {'Brightness': 94.17919921875, 'Sharpness': 46.02980041503906}
    
# -- selfie quality --
'Quality': {'Brightness': 93.77082824707031,'Sharpness': 20.927310943603516}
```

In [ ]:
# -- Selfie Image -- 
response = rek_client.detect_faces(Image={'S3Object':{
    'Bucket':bucket_name,
    'Name':selfie_image}},
                                   Attributes=['ALL'])
response

## Step 5. Onboarding 1:1 Verification with CompareFaces
-----

Compares a face in the source input image with each of the 100 largest faces detected in the target input image. in our case the source image will be the drivers license and the target image will be the selfie. 

In response, the operation returns an array of face matches ordered by similarity score in descending order. For each face match, the response provides a bounding box of the face, facial landmarks, pose details (pitch, roll, and yaw), quality (brightness and sharpness), and confidence value (indicating the level of confidence that the bounding box contains a face). The response also provides a similarity score, which indicates how closely the faces match.


<div class="alert alert-info"><strong> Note </strong>
    
Take a look at the FaceDetails of each face detected. it will return several helpful details of image quality for each face in the image. 

- SimilarityThreshold - The minimum level of confidence in the face matches that a match must meet to be included in the FaceMatches array. 

- QualityFilter - A filter that specifies a quality bar for how much filtering is done to identify faces. Filtered faces aren't compared. If you specify AUTO, Amazon Rekognition chooses the quality bar. If you specify LOW, MEDIUM, or HIGH, filtering removes all faces that do not meet the chosen quality bar. 

</div>


<div class="alert alert-info"><strong> Guidance </strong>
    
For identity verification use cases, it is recommended to set the SimilarityThreshold (for CompareFaces API) and FaceMatchThreshold (for SearchFacesByImage API) parameters to a value of 99%. This will improve the face match precision, and reduce the chances of a false positive (false match). However, it may lower the recall.  

Additionally, setting the QualityFilter to HIGH will help eliminate low quality images from consideration.


</div>


In [ ]:
response = rek_client.compare_faces(
    SourceImage={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': id_image,
        }
    },
    TargetImage={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': selfie_image,
        }
    },
    SimilarityThreshold = 99,
    QualityFilter='HIGH'
)
        
print(json.dumps(response, indent=3))  
    

## Step 6. system checks selfie against known onboarded users 
----
Here we simply search the onboarded user collection to see if the user has previously been onboarded. 



In [ ]:
image_external_id = "Sample_Jane_Maria"

response = rek_client.search_faces_by_image(
    CollectionId=collection_name,
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': selfie_image,
        }
    },
    MaxFaces=1,
    FaceMatchThreshold=90,
    QualityFilter='AUTO'
)

try:
    external_image_id = response["FaceMatches"][0]["Face"]["ExternalImageId"]
    similarity_score =  response["FaceMatches"][0]["Similarity"]
    face_id = response["FaceMatches"][0]["Face"]["FaceId"]
    print("-- Face Found in   : {}".format(collection_name))
    print("-- FaceID          : {} ".format(face_id))
    print("-- Similarity      : {} ".format(similarity_score))
    print("-- ExternalImageID : {} \n".format(external_image_id))
except:
    print("-- Face NOT Found in   : {}".format(collection_name))
    print(json.dumps(response, indent=3))  


response = rek_client.search_faces_by_image(
    CollectionId=fraud_collection_name,
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': selfie_image,
        }
    },
    MaxFaces=1,
    FaceMatchThreshold=90,
    QualityFilter='AUTO'
)

try:
    external_image_id = response["FaceMatches"][0]["Face"]["ExternalImageId"]
    similarity_score =  response["FaceMatches"][0]["Similarity"]
    face_id = response["FaceMatches"][0]["Face"]["FaceId"]
    print("-- Face Found in   : {}".format(fraud_collection_name))
    print("-- FaceID          : {} ".format(face_id))
    print("-- Similarity      : {} ".format(similarity_score))
    print("-- ExternalImageID : {} \n".format(external_image_id))

except:
    print("-- Face NOT Found in   : {}".format(fraud_collection_name))
    print(json.dumps(response, indent=3))  


## Step 7. system indexes user's trusted document and potentially selfie 
----
Here we simply index the faces on the drivers license and optionally index the selfie image as well. Why index both? well typically the image on the drivers license should be somewhat older than the selfie and having two images is better than a single when searching. 


In [ ]:
# Index a drivers license face 
print("indexing drivers license")
response = rek_client.index_faces(
    CollectionId=collection_name,
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': selfie_image,
        }
    },
    ExternalImageId=image_external_id,
    DetectionAttributes=[
        'DEFAULT',
    ],
    MaxFaces=1, # how many faces to detect 
    QualityFilter='HIGH' # apply quality filter 
)

try:
    face_id = response['FaceRecords'][0]['Face']['FaceId']
    print("-- FaceID : {} --\n".format(face_id))
    print(json.dumps(response, indent=3))
except:
    print(json.dumps(response, indent=3))
    
print("indexing selfie image  ")
# Index a selfie image
response = rek_client.index_faces(
    CollectionId=collection_name,
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': selfie_image,
        }
    },
    ExternalImageId=image_external_id,
    DetectionAttributes=[
        'DEFAULT',
    ],
    MaxFaces=1, # how many faces to detect 
    QualityFilter='HIGH' # apply quality filter 
)

try:
    face_id = response['FaceRecords'][0]['Face']['FaceId']
    print("-- FaceID : {} --\n".format(face_id))
    print(json.dumps(response, indent=3))
except:
    print(json.dumps(response, indent=3))

## Clean up
----
Here we simply need to delete our collections 


In [ ]:
# onboarded users collection

try:
    rek_client.delete_collection(
        CollectionId=collection_name
    )
except:
    print("collection: {} NOT FOUND".format(collection_name))

# fraudulent user collection 

try:
    rek_client.delete_collection(
        CollectionId=fraud_collection_name
    )
except:
     print("collection: {} NOT FOUND".format(fraud_collection_name))